In [25]:
import numpy as np

In [26]:
# Pre-SVM: Margin-based linear classifier with explicit norm control
# This isnt actual primal SVM but bridging from margin perceptron to it

# Goal:
#   Learn a linear separator that balances correctness -- margin violations
#   against robustness  -- sensitivity of output to input perturbations
# The concept involved is updation of w with two constraints : Correctness and Robustness

# Decision rule remains sign(w·x + b)
# Large ||w|| => high sensitivity (fragile classifier)
# Shrinking ||w|| flattens decision surface => robustness
# Updates respond both margin violations and norm growth
# You now want to change w in the direction that most rapidly reduces a penalty for violating the margin while also resisting instability.
# Introduce dw  -> direction in parameter space that improves on both points locally

# This is NOT yet a true SVM:
#  No single scalar objective is optimized
#  No guarantee of convergence
#  Serves to expose why hinge loss and convex optimization are required

In [27]:
def pre_svm(X,Y,step_size,stab_coeff,tol=1e-5):
    n_samples, n_features = X.shape
    cycles_func = 0
    w_prev = np.zeros(n_features)
    w = np.zeros(n_features)
    b = 0.0
    converged = False
    while not converged:
        w_prev = w.copy()
        cycles_func += 1
        for i in range (n_samples):
            margin = Y[i] * ((w @ X[i]) + b)
            w_stab = -stab_coeff*w
            w_margin = Y[i]*X[i] if margin<1 else 0
            b_margin = Y[i] if margin<1 else 0

            w += step_size * (w_margin + w_stab)
            b += step_size * b_margin

        change = np.linalg.norm(w - w_prev)
        if change < tol or cycles_func>2000: 
            converged = True


    norm_w = np.linalg.norm(w)
    gamma = np.min(Y * (X @ w + b)) /norm_w if norm_w !=0 else 0
    R = np.max(np.linalg.norm(X,axis=1))

    return R,gamma,cycles_func,w
    




In [28]:
# 1st dataset 
X = np.array([
    [1.0, 1.0],
    [1.2, 0.8],
    [0.8, 1.3],
    [1.1, 1.4],
    [3.0, 3.1],
    [2.8, 2.9],
    [3.2, 2.7],
    [3.1, 3.3]
])

Y = np.array([-1, -1, -1, -1, 1, 1, 1, 1])
step_size = 0.01
stab_coeff = 0.1

r1_r,r1_g,r1_c,r1_w = pre_svm(X,Y,step_size,stab_coeff)
print("================= PRE SVM  1 ====================")
print()
print(f"Cycles : {r1_c} | Gamma : {r1_g} | R : {r1_r} | ||w|| : {np.linalg.norm(r1_w)}")

================= PRE SVM  1 ====================

Cycles : 2001 | Gamma : 1.1245730849522488 | R : 4.527692569068709 | ||w|| : 0.8916852873067274


In [29]:
# 2nd dataset
X = np.array([
    [1.0, 1.0],
    [1.2, 0.9],
    [0.9, 1.2],
    [1.1, 1.0],
    [2.0, 2.0],   # noisy point
    [3.0, 3.1],
    [2.8, 2.9],
    [3.2, 3.0]
])

Y = np.array([-1, -1, -1, 1, -1, 1, 1, 1])
r2_r,r2_g,r2_c,r2_w = pre_svm(X,Y,step_size,stab_coeff)
print("=================   PRE SVM 2  ====================")
print()
print(f"Cycles : {r2_c} | Gamma : {r2_g} | R : {r2_r} | ||w|| : {np.linalg.norm(r2_w)}")

=================   PRE SVM 2  ====================

Cycles : 2001 | Gamma : -1.185256051541581 | R : 4.386342439892262 | ||w|| : 0.832575933220935
